In [ ]:
from rmgpy.molecule.molecule import *
from rmgpy import settings
from rmgpy.species import *
from rmgpy.data.rmg import RMGDatabase
import pubchempy as pcp
from collections import Counter
import pandas as pd
import ast
from scipy.optimize import minimize
import numpy as np
import time

# S.1.1.3 Thermodynamic Parameter Extraction

1. [Loading](#loading-thermodynamic-databases)
2. [Getting Molecule Information](#get-molecule-info)
3. [Helper Functions to Solve for Stoichiometry](#helper-functions-to-solve-for-stoichiometry)
4. [Getting Thermodynamic Data](#get-thermodynamic-data)
5. [Applying to All Files](#applying-to-all-files)


#### Loading Thermodynamic Databases

In [2]:
database = RMGDatabase()

database.load(
            path = settings['database.directory'],
            thermo_libraries = ['Klippenstein_Glarborg2016', 'BurkeH2O2', 'thermo_DFT_CCSDTF12_BAC', 
                               'DFT_QCI_thermo',
                           'primaryThermoLibrary', 'primaryNS', 'NitrogenCurran', 'NOx2018', 'FFCM1(-)',
'SulfurLibrary', 'SulfurGlarborgH2S','SABIC_aromatics',],
            transport_libraries = [],
            reaction_libraries = [],
            seed_mechanisms = [],
            kinetics_families = [],
            kinetics_depositories = ['training'],
            depository = False, # Don't bother loading the depository information, as we don't use it
        )

#### Get Molecule Info

This function uses PubChemPy to convert from CAS number to SMILES, which is what RMG uses. Additionally, it yields element count to allow us to solve for stoichiometry later.

In [18]:
def get_molecule_info(cas_number):
    try:
        # Fetch the compound using the CAS number
        compound = pcp.get_compounds(cas_number, 'name', domain='compound')

        if compound:
            compound = compound[0]  # Get the first compound result
            smiles = compound.canonical_smiles  # Get the SMILES string
            
            # Fetch the elements using the 'elements' property
            elements = compound.elements  # List of elements in the compound
            
            # Count elements in the molecular formula using the 'elements' property
            element_counts = Counter(elements)

            return {
                'CAS': cas_number,
                'SMILES': smiles,
                'Element_Count': dict(element_counts),
            }
        else:
            return None

    except Exception as e:
        print(f"Error occurred: {e}")
        return None
    

#### Helper Functions to Solve For Stoichiometry

Many patents do not include stoichiometric coefficients, making it challenging to calculate enthalpy of reaction. However, we can define two functions to help us optimize the stoichiometric coefficients to reflect reality as nearly as possible.

`total_element_count` returns the count of all reactant elements.

`objective function` is ts the function that can optimize the `total_element_count`. In it, we compute the sum of squared differences across each element for reactants vs. products. Then, we add a penalty if the carbon is more than 5% off.

In [19]:
def total_element_count(coefficients, reactants, all_elements):
    total = Counter()
    
    # Sum the scaled reactants and reagents element counts
    for coeff, react in zip(coefficients, reactants):
        for element in all_elements:
            total[element] += coeff * react.get(element, 0)
    
    return total

def objective_function(coefficients, product_elements, reactants_and_reagents, all_elements):
    total = total_element_count(coefficients, reactants_and_reagents, all_elements)
    
    # Calculate the sum of squared differences between total and product elements
    squared_diffs = 0
    for element in all_elements:
        squared_diffs += (total[element] - product_elements[element]) ** 2
    
    # Apply heavy penalty if carbon ('C') is not within 5% of the target
    target_carbon = product_elements.get('C', 0)
    total_carbon = total.get('C', 0)
    
    # Check if the total carbon is outside of 5% tolerance
    if abs(total_carbon - target_carbon) > 0.01 * target_carbon:
        penalty_weight = 1000  # Heavily penalize if carbon is not matched within 5%
        squared_diffs += penalty_weight * abs(total_carbon - target_carbon)

    return squared_diffs

In [20]:

def convert_to_list(x):
    try:
        # Attempt to evaluate the string as a Python literal
        return ast.literal_eval(x) if isinstance(x, str) else x
    except (ValueError, SyntaxError):
        # If it's not a stringified list, return the value as is
        return x
    


#### Get Thermodynamic Data

This function first collects all needed information for reactants/reagants in `react_reag` which holds all information, and `reactants` which holds element counts only. The same is done for products, where `products` holds complete product information and `product_elements` holds element information.

`react_reag` and `products` are then made into dataframes. Then, the stoichiometry is optimized. RMG is then queried for each reactant participant, and the enthalpy and entropy are added on to a running total multiplied by their stoichiometric coefficient.

In [21]:
def get_thermo(entry):
    products = []
    product_elements = Counter()

    if not isinstance(entry.products, list) or not isinstance(entry.reactants, list):
        return [np.nan,np.nan]
    
    for cas in entry.products:
        info = get_molecule_info(cas)
        if info is not None:
            products.append(info)
            product_elements = product_elements + Counter(info['Element_Count'])

    products = pd.DataFrame(products)

    react_reag = []
    reactants = []
    for cas in entry.reagant_cas:
        info = get_molecule_info(cas)
        if info is not None:
            react_reag.append(info)
            reactants.append(Counter(info['Element_Count']))
    for cas in entry.reactants:
        info = get_molecule_info(cas)
        if info is not None:
            react_reag.append(info)
            reactants.append(Counter(info['Element_Count']))

    react_reag = [x for x in react_reag if x is not None]
    react_reag = pd.DataFrame(react_reag)

    all_elements = list(product_elements.keys())  # ['C', 'H', 'O']
    if len(reactants) == 0:
        return [np.nan,np.nan]


    # Initial guess for the coefficients (start with 1 for each reactant/reagent)
    initial_guess = np.ones(len(reactants))

    # Perform the optimization to minimize the objective function
    result = minimize(objective_function, initial_guess, args=(product_elements, reactants, all_elements), bounds=[(0, None)] * len(reactants))

    # Extract the optimized coefficients
    optimal_coefficients = result.x
    react_reag['stoich_coeff']=optimal_coefficients*-1

    entropy_rxn = 0
    enthalpy_rxn = 0

    for index, reactant in react_reag.iterrows():
        try:
            spc = Species(smiles=reactant['SMILES'])
            if spc is not None:
                spc.get_thermo_data()

                entropy_rxn = entropy_rxn + spc.get_entropy(298)*reactant.stoich_coeff
                enthalpy_rxn = enthalpy_rxn + spc.get_enthalpy(298)*reactant.stoich_coeff
        except Exception as e:
            print(f"Error processing product at index {index}: {e}")
            return [np.nan,np.nan]

    for index, prod in products.iterrows():
        try:
            spc = Species(smiles=prod['SMILES'])
            if spc is not None:
                spc.get_thermo_data()

                entropy_rxn = entropy_rxn + spc.get_entropy(298)
                enthalpy_rxn = enthalpy_rxn + spc.get_enthalpy(298)
        except Exception as e:
            print(f"Error processing product at index {index}: {e}")
            return [np.nan,np.nan]
    
    return [entropy_rxn, enthalpy_rxn]

#### Applying to all Files

Now, this function is applied to all files that have not previously been processed. 

We note that because this is a gas phase kinetics resource, many heavy elements and all salts lack sufficient information to calculate reaction thermodynamics. Overall, approximately 1000 patents did not have calculable kinetics for one reason or another.

Additionally, RMG works by first searching its thermodynamic databases for an entry. If it finds none, it attempts to calculate the value.

In [22]:
def list_files_in_folder(folder_path):
    # List all files in the specified folder
    file_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    return file_names


folder_path = '../parsed_dataframes'
files = list_files_in_folder(folder_path)

for file in files:
    filename = '../parsed_dataframes/'+file
    print(file)
    df = pd.read_excel(filename).drop(columns = ["Unnamed: 0"])
    df = df.applymap(convert_to_list)

    if 'products' not in df.columns:
        print('no rdf yet')
        continue
    if 'enthalpy_rxn' in df.columns:
        print('already calculated')
        continue

    df["enthalpy_rxn"] = pd.Series(dtype = float)
    df["entropy_rxn"] = pd.Series(dtype = float)
    for index, reaction in df.iterrows():
        thermo_data = get_thermo(reaction)
        df.loc[index, "enthalpy_rxn"] = thermo_data[1]
        df.loc[index, "entropy_rxn"] = thermo_data[0]
    df.to_excel(filename)
    time.sleep(1.5)



100-21-0.xlsx
already calculated
100-41-4.xlsx
already calculated
100-42-5.xlsx
already calculated
101-68-8.xlsx
already calculated
102-71-6.xlsx
already calculated
104-76-7.xlsx
already calculated
106-42-3.xlsx
already calculated
106-89-8.xlsx
already calculated
106-97-8.xlsx
already calculated
106-98-9.xlsx
already calculated
106-99-0.xlsx
already calculated
107-02-8.xlsx
already calculated
107-06-2.xlsx
already calculated
107-13-1.xlsx
already calculated
107-15-3.xlsx
already calculated
107-18-6.xlsx
already calculated
107-21-1.xlsx
already calculated
107-83-5.xlsx
already calculated
108-05-4.xlsx
already calculated
108-31-6.xlsx
already calculated
108-38-3.xlsx
already calculated
108-88-3.xlsx
already calculated
108-93-0.xlsx
already calculated
108-94-1.xlsx
already calculated
108-95-2.xlsx
already calculated
109-99-9.xlsx
already calculated
110-63-4.xlsx
already calculated
110-82-7.xlsx
already calculated
110-88-3.xlsx
already calculated
111-42-2.xlsx
already calculated
111-66-0.x